In [3]:
import pandas as pd
import json

# Load the monthly tone averages
try:
    tone_data = pd.read_csv("../data/gdelt_monthly_tone_averages.csv")
    print(f"Loaded tone data with {len(tone_data)} records")
except FileNotFoundError:
    print("Tone data file not found. Please run the tone analysis script first.")
    exit(1)

# Calculate the average tone across all sources for each month/year
monthly_avg = tone_data.groupby(['year', 'month'])['average_tone'].mean().reset_index()
print(f"Calculated {len(monthly_avg)} monthly averages across all sources")

# Find the min and max tone values for scaling
min_tone = monthly_avg['average_tone'].min()
max_tone = monthly_avg['average_tone'].max()
print(f"Tone range: {min_tone:.2f} to {max_tone:.2f}")

# Create a nested dictionary structure for easy access in JavaScript
tone_by_year_month = {}

for _, row in monthly_avg.iterrows():
    year = int(row['year'])
    month = int(row['month'])
    tone = float(row['average_tone'])
    
    # Create year entry if it doesn't exist
    if year not in tone_by_year_month:
        tone_by_year_month[year] = {}
    
    # Add month entry
    tone_by_year_month[year][month] = tone

# Also create individual source dictionaries for reference
source_tone_data = {}

for source in tone_data['source'].unique():
    source_data = tone_data[tone_data['source'] == source]
    source_dict = {}
    
    for _, row in source_data.iterrows():
        year = int(row['year'])
        month = int(row['month'])
        tone = float(row['average_tone'])
        
        if year not in source_dict:
            source_dict[year] = {}
        
        source_dict[year][month] = tone
    
    source_tone_data[source] = source_dict

# Create a combined data structure
calendar_data = {
    'avg_tone': tone_by_year_month,
    'source_tone': source_tone_data,
    'metadata': {
        'min_tone': min_tone,
        'max_tone': max_tone
    }
}

# Save as JavaScript file that can be included directly
with open("../data/tone_data.js", "w") as f:
    f.write("// Generated tone data for calendar visualization\n")
    f.write("const toneData = ")
    f.write(json.dumps(calendar_data, indent=2))
    f.write(";\n")

print("Tone data saved to website/scripts/tone_data.js")

# Also save as JSON file for reference
with open("../data/tone_data.json", "w") as f:
    json.dump(calendar_data, f, indent=2)

print("Tone data also saved to website/data/tone_data.json")

Loaded tone data with 369 records
Calculated 123 monthly averages across all sources
Tone range: -3.53 to -1.93
Tone data saved to website/scripts/tone_data.js
Tone data also saved to website/data/tone_data.json
